### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [907]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, 
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances = []
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return 1 - np.sum((l_c**2 / l_s + r_c**2 / r_s) / (l_s + r_s), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c * np.log2((l_c) / l_s) + r_c * np.log2((r_c ) / r_s), axis=1)/ (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(r_c / (l_s + r_s), axis=1)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.sqrt(n_feature)])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.log2(n_feature)])

    def __get_feature_ids_N(self, n_feature):
        return np.random.permutation(n_feature)
    
    def __sort_samples(self, x, y): 
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        
        features = self.get_feature_ids(x.shape[1])
        min_I = 2
        res_ind = -1
        
        for f in features:
            x_sorted, y_sorted = self.__sort_samples(x[:, f], y)
            
            thrs = np.where(y_sorted[:-1] != y_sorted[1:])[0] + 1
            class_len = (thrs - np.hstack((np.array([0]), thrs[:-1])))
            class_len = class_len.reshape(-1, 1)
            class_cnt = np.zeros((thrs.shape[0], self.num_class))
            class_cnt[np.arange(thrs.shape[0]), y_sorted[thrs - 1]] = 1
            l_c = np.cumsum(class_cnt*class_len, axis=0).astype(int)
            r_c = np.bincount(y_sorted, minlength=self.num_class) - l_c
            l_s = np.sum(l_c, axis=1).reshape(-1, 1)
            r_s = np.sum(r_c, axis=1).reshape(-1, 1)

            I = self.G_function(l_c, l_s, r_c, r_s)
            ind = np.argmin(I)
            
            min_f = l_s[ind][0]
            if np.min(I) < min_I:
                res_ind = f
                res_thr = x_sorted[min_f]
                min_I = np.min(I)            
        
        initial_impurity = self.G_function(np.expand_dims(np.bincount(y, minlength=self.num_class),axis=0),
                                           y.shape[0], 0, 0.001)        
        impurity_decrease = initial_impurity - min_I
        if self.feature_importances == []:
            self.feature_importances = np.array([0]*x.shape[1]).astype(float)        
        self.feature_importances[res_ind] += impurity_decrease / x.shape[0]
        return res_ind, res_thr
            
    def __fit_node(self, x, y, node_id, depth):
        
        if depth == self.max_depth or len(y) <= self.min_samples_split or np.unique(y).shape[0] == 1:
            self.tree[node_id] = (self.__class__.LEAF_TYPE, np.argmax(np.bincount(y)),
                                  np.bincount(y) / y.shape[0])
            return 

        feature, threshold = self.__find_threshold(x, y)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              feature, threshold)

        X_l, X_r, y_l, y_r = self.__div_samples(x, y, feature, threshold)

        if X_l.shape[0] == 0 or X_r.shape[0] == 0:
            cnt = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, cnt.argmax(),
                                  cnt.astype(float) / y.shape[0])
            return
        
        self.__fit_node(X_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        x = np.asarray(x).astype(float)
        y = np.asarray(y).astype(int)
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id): 
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1] 

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X): 
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

## Проверка скорости работы на wine

In [908]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [909]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [910]:
%time clf.fit(X_train, y_train)

CPU times: user 2.57 ms, sys: 38 µs, total: 2.61 ms
Wall time: 1.42 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [911]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.4 ms, sys: 3.94 ms, total: 22.3 ms
Wall time: 20.9 ms


/usr/lib/python3/dist-packages/ipykernel_launcher.py:98: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


## Проверка качества работы на wine

In [912]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [913]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [923]:
data = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")
data = data[['iid', 'gender', 'pid', 'match', 'int_corr', 'race', 'goal', 'date',
       'go_out', 'career_c', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1',
       'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1',
       'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1',
       'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1',
       'intel3_1', 'amb3_1']]

data.fillna(-1000)

male = data.loc[data.gender == 1].drop_duplicates(subset=['pid','iid'])
male = male.drop('gender', axis=1).dropna()
female = data.loc[data.gender == 0].drop_duplicates(subset=['iid'])
female = female.drop(['gender', 'match', 'int_corr'], axis=1).dropna()



male.columns = male.columns + '_m'
female.columns = female.columns + '_f'
female = female.drop(['pid_f'], axis=1)
pairs = male.join(female.set_index('iid_f'), on='pid_m', how='inner')
pairs = pairs.drop(['iid_m', 'pid_m'], axis=1)

X = pairs.drop('match_m', axis=1).values
y = pairs.match_m.values

In [924]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [925]:
%time clf.fit(X_train, y_train)

CPU times: user 61.4 ms, sys: 15 µs, total: 61.4 ms
Wall time: 60.3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [926]:
%time my_clf.fit(X_train, y_train)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:98: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


CPU times: user 491 ms, sys: 0 ns, total: 491 ms
Wall time: 489 ms


## Проверка качества работы на Speed Dating Data

In [927]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.539319248826291

In [928]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5475887787100602

## Задание 3

## Задание 4

In [929]:
print(pairs.columns[np.argsort(my_clf.feature_importances)][:10])

Index(['attr1_1_f', 'intel4_1_m', 'sinc4_1_m', 'go_out_f', 'exphappy_f',
       'fun2_1_m', 'intel1_1_f', 'shar4_1_m', 'fun3_1_f', 'intel1_1_m'],
      dtype='object')


In [930]:
print(pairs.columns[np.argsort(clf.feature_importances_)][:10])

Index(['exphappy_f', 'sinc4_1_f', 'sinc2_1_f', 'shar2_1_f', 'sinc3_1_m',
       'intel4_1_f', 'race_f', 'fun2_1_m', 'race_m', 'fun1_1_m'],
      dtype='object')


## Задание 5

In [931]:
from sklearn.model_selection import GridSearchCV
params={
    'criterion' : ['gini',  'entropy'],
    'min_samples_split' : [2,5,10],
    'max_features' : ['sqrt', 'log2'],
    'max_depth' : np.linspace(500, 5000, 10)
}

In [932]:
r_forest = RandomForestClassifier(n_estimators=5)
rf = GridSearchCV(r_forest, params)
rf.fit(X_train, y_train)
print(rf.best_params_)
print(rf.best_score_)

/home/ayd98/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'criterion': 'gini', 'max_depth': 2000.0, 'max_features': 'log2', 'min_samples_split': 10}
0.8358738036157392
